In [1]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, array_to_img, img_to_array, load_img
import os
import random
import math
import shutil

In [2]:
ORIGINAL_DATA_PATH = './data/original_images'
AUGMENTED_DATA_PATH = './data/augmented_images'
BALANCED_DATA_PATH = './data/balanced_images'
SPLIT_DATA_PATH = './data/images'

CATEGORY_DIRECTORIES = ['Chickenpox', 'Cowpox', 'Healthy', 'HFMD', 'Measles', 'Monkeypox']

TRAIN_RATIO = 0.8
TEST_RATIO = 0.1
VAL_RATIO = 0.1

TRAIN_DIRECTORY = 'train'
TEST_DIRECTORY = 'test'
VAL_DIRECTORY = 'val'

SUB_DIRECTORIES = [TRAIN_DIRECTORY, TEST_DIRECTORY, VAL_DIRECTORY]

DS_STORE = '.DS_Store'

TARGET_COUNT = 1000

ROTATION_RANGE = 90
WIDTH_SHIFT_RANGE = 0.2
HEIGHT_SHIFT_RANGE = 0.2
SHEAR_RANGE = 0.2
ZOOM_RANGE = 0.2
HORIZONTAL_FLIP = True
FILL_MODE = 'nearest'

In [3]:
def count_images(dir_name):
    total = 0
    for category in CATEGORY_DIRECTORIES:
        path = os.path.join(dir_name, category)
        images = [img for img in os.listdir(path) if img.endswith(('jpg','jpeg'))]
        print(f"Number of images in {path.split('/')[-1]}: {len(images)}")
        total += len(images)
    print(f"Total image count: {total}")

In [4]:
count_images(ORIGINAL_DATA_PATH)

Number of images in Chickenpox: 75
Number of images in Cowpox: 66
Number of images in Healthy: 114
Number of images in HFMD: 161
Number of images in Measles: 55
Number of images in Monkeypox: 284
Total image count: 755


In [5]:
def copy_image(old_path, copy_path):
    if old_path.endswith(('jpg','jpeg')):
        name, ext = os.path.splitext(old_path)
        new_name = f"{name}_ORIGINAL{ext}".split("/")[-1]
        new_path = os.path.join(copy_path, new_name)
        # copy the images with ORIGINAL appended
        shutil.copy(old_path, new_path)
        img_name = old_path.split("/")[-1]
        print(f"Renamed: {img_name} → {new_name}")

In [6]:
datagen = ImageDataGenerator(
    rotation_range=ROTATION_RANGE,
    width_shift_range=WIDTH_SHIFT_RANGE,
    height_shift_range=HEIGHT_SHIFT_RANGE,
    shear_range=SHEAR_RANGE,
    zoom_range=ZOOM_RANGE,
    horizontal_flip=HORIZONTAL_FLIP,
    fill_mode=FILL_MODE)

In [7]:
for category in CATEGORY_DIRECTORIES:
    path = AUGMENTED_DATA_PATH
    path = os.path.join(path, category)
    os.mkdir(path)

In [8]:
for category in CATEGORY_DIRECTORIES:
    path = BALANCED_DATA_PATH
    path = os.path.join(path, category)
    os.mkdir(path)

In [9]:
for dataset in SUB_DIRECTORIES:
    for category in CATEGORY_DIRECTORIES:
        path = os.path.join(SPLIT_DATA_PATH, dataset, category)
        os.mkdir(path)

In [10]:
for category in CATEGORY_DIRECTORIES:
    path_o = os.path.join(ORIGINAL_DATA_PATH, category)
    path_a = os.path.join(AUGMENTED_DATA_PATH, category)
    files = os.listdir(path_o)
    for index, file in enumerate(files):
        if DS_STORE in file: ### Skip .DS_Store file
            continue
        print(f'{path_o}/{file}') ### Print the pathname and file name to get an idea about the processing file
        img = load_img(str(os.path.join(path_o, file))) ### This is a PIL image
        x = img_to_array(img) ### This is a NumPy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape) ### This is a NumPy array with shape (1, 3, 150, 150)
        ### The .flow() command generates batches of randomly transformed images and save the results in the 
        ### ./data/augmented_images/<data_directory> along with the original image
        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=path_a, save_prefix=file.split(".")[0], save_format='jpg'):
            i += 1
            if i > 15:
                break ### Stop looping the generator infinitely
        ### Save the original image in ./data/augmented/<data_directory> as well
        copy_image(os.path.join(path_o, file), path_a)

./data/original_images/Chickenpox/CHP_05_01.jpg
Renamed: CHP_05_01.jpg → CHP_05_01_ORIGINAL.jpg
./data/original_images/Chickenpox/CHP_17_01.jpg
Renamed: CHP_17_01.jpg → CHP_17_01_ORIGINAL.jpg
./data/original_images/Chickenpox/CHP_09_01.jpg
Renamed: CHP_09_01.jpg → CHP_09_01_ORIGINAL.jpg
./data/original_images/Chickenpox/CHP_21_01.jpg
Renamed: CHP_21_01.jpg → CHP_21_01_ORIGINAL.jpg
./data/original_images/Chickenpox/CHP_42_01.jpg
Renamed: CHP_42_01.jpg → CHP_42_01_ORIGINAL.jpg
./data/original_images/Chickenpox/CHP_33_01.jpg
Renamed: CHP_33_01.jpg → CHP_33_01_ORIGINAL.jpg
./data/original_images/Chickenpox/CHP_50_01.jpg
Renamed: CHP_50_01.jpg → CHP_50_01_ORIGINAL.jpg
./data/original_images/Chickenpox/CHP_15_01.jpg
Renamed: CHP_15_01.jpg → CHP_15_01_ORIGINAL.jpg
./data/original_images/Chickenpox/CHP_19_01.jpg
Renamed: CHP_19_01.jpg → CHP_19_01_ORIGINAL.jpg
./data/original_images/Chickenpox/CHP_07_01.jpg
Renamed: CHP_07_01.jpg → CHP_07_01_ORIGINAL.jpg
./data/original_images/Chickenpox/CHP_52

In [11]:
count_images(AUGMENTED_DATA_PATH)

Number of images in Chickenpox: 1275
Number of images in Cowpox: 1122
Number of images in Healthy: 1937
Number of images in HFMD: 2735
Number of images in Measles: 935
Number of images in Monkeypox: 4824
Total image count: 12828


In [12]:
def create_balanced_dataset():
    for category in CATEGORY_DIRECTORIES:
        source_path = os.path.join(AUGMENTED_DATA_PATH, category)
        target_path = os.path.join(BALANCED_DATA_PATH, category)

        images = [img for img in os.listdir(source_path) if img.endswith(("jpg", "jpeg"))]

        original_images = [img for img in images if "ORIGINAL" in img]
        other_images = [img for img in images if "ORIGINAL" not in img]

        selected_images = original_images[:min(TARGET_COUNT, len(original_images))]

        if len(selected_images) < TARGET_COUNT:
            remaining_needed = TARGET_COUNT - len(selected_images)
            selected_images.extend(random.sample(other_images, min(remaining_needed, len(other_images))))

        for img in selected_images:
            shutil.copy(os.path.join(source_path, img), os.path.join(target_path, img))

        print(f"{category}: {len(selected_images)} images copied to {target_path}")
    print("\n Balanced dataset created successfully!\n")

In [13]:
create_balanced_dataset()

Chickenpox: 1000 images copied to ./data/balanced_images/Chickenpox
Cowpox: 1000 images copied to ./data/balanced_images/Cowpox
Healthy: 1000 images copied to ./data/balanced_images/Healthy
HFMD: 1000 images copied to ./data/balanced_images/HFMD
Measles: 935 images copied to ./data/balanced_images/Measles
Monkeypox: 1000 images copied to ./data/balanced_images/Monkeypox

 Balanced dataset created successfully!



In [14]:
count_images(BALANCED_DATA_PATH)

Number of images in Chickenpox: 1000
Number of images in Cowpox: 1000
Number of images in Healthy: 1000
Number of images in HFMD: 1000
Number of images in Measles: 935
Number of images in Monkeypox: 1000
Total image count: 5935


In [15]:
def split_dataset():
    for category in CATEGORY_DIRECTORIES:
        source_path = os.path.join(BALANCED_DATA_PATH, category)

        # Define split directories
        train_path = os.path.join(SPLIT_DATA_PATH, TRAIN_DIRECTORY, category)
        val_path = os.path.join(SPLIT_DATA_PATH, VAL_DIRECTORY, category)
        test_path = os.path.join(SPLIT_DATA_PATH, TEST_DIRECTORY, category)

        images = [img for img in os.listdir(source_path) if img.endswith(('jpg', 'jpeg'))]
        random.shuffle(images)
            
        # Split images
        train_split = int(len(images) * TRAIN_RATIO)
        val_split = int(len(images) * (TRAIN_RATIO + VAL_RATIO))

        train_images = images[:train_split]
        val_images = images[train_split:val_split]
        test_images = images[val_split:]

        # Copy images to respective folders
        for img in train_images:
            shutil.copy(os.path.join(source_path, img), os.path.join(train_path, img))

        for img in val_images:
            shutil.copy(os.path.join(source_path, img), os.path.join(val_path, img))

        for img in test_images:
            shutil.copy(os.path.join(source_path, img), os.path.join(test_path, img))

        print(f"{category}: {len(train_images)} train, {len(val_images)} val, {len(test_images)} test images")

    print("\n Dataset successfully split into train, val, and test. \n")

In [16]:
split_dataset()

Chickenpox: 800 train, 100 val, 100 test images
Cowpox: 800 train, 100 val, 100 test images
Healthy: 800 train, 100 val, 100 test images
HFMD: 800 train, 100 val, 100 test images
Measles: 748 train, 93 val, 94 test images
Monkeypox: 800 train, 100 val, 100 test images

 Dataset successfully split into train, val, and test. 

